In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np 
import rasterio as rs
from rasterio.plot import show
from pyproj import CRS
import geopy
import rioxarray


from matplotlib import pyplot as plt
%matplotlib inline

/home/arfa/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
shapefile = gpd.read_file("./DTM/DHMVII_vdc_k01.shp")
shapefile

,Id,OpnDatum1,OpnDatum2,geometry
0,0,2013-02-20,None,"MULTIPOLYGON Z (((162079.010 238604.770 0.000,..."
1,0,2013-04-01,None,"POLYGON Z ((148803.300 243432.780 0.000, 14882..."
2,0,2013-02-20,2013-04-01,"MULTIPOLYGON Z (((156554.470 243736.800 0.000,..."


In [3]:
shapefile.crs

<Projected CRS: PROJCS["Belge 1972 / Belgian Lambert 72",GEOGCS["B ...>
Name: Belge 1972 / Belgian Lambert 72
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: Reseau National Belge 1972
- Ellipsoid: International 1924
- Prime Meridian: Greenwich

In [4]:
dsm = rs.open(r'DSM_k01.tif')
show(dsm)

RasterioIOError: DSM_k01.tif: No such file or directory

In [ ]:
dtm = rs.open(r'DTM_k01.tif')
show(dtm)

In [ ]:
dsm_array = dsm.read(1)
dtm_array = dtm.read(1)

In [ ]:
chm_array = dsm_array - dtm_array

In [ ]:
chm_array

In [ ]:
show(chm_array)

In [ ]:
slice_ar = chm_array[8000:8010,10100:10120]
slice_ar

#import earthpy as et
#import earthpy.plot as ep
#ep.plot_bands(slice_ar,cmap='viridis')
#plt.show()

In [ ]:
nx = slice_ar.shape[1]
ny = slice_ar.shape[0]
x = range(nx)
y = range(ny)

In [ ]:
size = plt.figure(figsize = (10,10))
ha = size.add_subplot(111, projection = "3d")

X,Y = np.meshgrid(x,y)
ha.plot_surface(X, Y, slice_ar)
plt.show()

In [ ]:
dsm.bounds

In [ ]:
dsm.meta

In [ ]:
import rioxarray as rx

In [ ]:
surface = rx.open_rasterio("DHMVIIDSMRAS1m_k01.tif")
surface

In [ ]:
surface.rio.crs

In [ ]:
surface.rio.bounds()

In [ ]:
surface.values

In [ ]:
from pyproj import CRS
epsg = surface.rio.crs.to_epsg()
crs = CRS(epsg)
crs

In [ ]:
surface = rioxarray.open_rasterio("DSM/DSM_k06.tif")
terrain = rioxarray.open_rasterio("DTM/DTM_k06.tif")

In [ ]:
surface.rio.crs.wkt

In [ ]:
terrain.rio.crs.wkt

In [ ]:
import rasterio

In [ ]:
df = rasterio.open('DSM_k07.tif')
df1 = rasterio.open("DTM_k07.tif")

In [ ]:
df.bounds
df1.bounds

In [ ]:
BoundingBox(left=145000.0, bottom=238000.0, right=162000.0, top=247000.0)

In [ ]:
df7 = rasterio.open("DTM_k06.tif")

In [ ]:
df7.bounds.left

In [ ]:
df7.meta

In [ ]:
import requests

In [ ]:
address =  "Spijker 29, 2910 Essen"
def details(address: str):
    req = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    info = {'address' : address, 
                'x_value' : req['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : req['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : req['LocationResult'][0]['Thoroughfarename'],
                'house_number' : req['LocationResult'][0]['Housenumber'], 
                'postcode': req['LocationResult'][0]['Zipcode'], 
                'municipality' : req['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']['coordinates']]
    
    return info

info = details(address)
info


In [ ]:
x = info["x_value"]
y = info["y_value"]